In [2]:
import cv2 as cv
import glob
import numpy as np

In [3]:
%matplotlib tk
import matplotlib.pyplot as plt
import numpy as np

from PIL import Image

In [4]:
left_image_path = 'data/test_left.JPG'
right_image_path = 'data/test_right.JPG'

im_left = cv.imread(left_image_path, 1)
im_right = cv.imread(right_image_path, 1)

criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 30, 0.001)

In [5]:
# idea here we need a chessboard to calibrate the camera, world_scaling can be the real size 
rows = 7 
cols = 7
world_scaling = 1

In [6]:
def find_carmera_matrix(image):
    objp = np.zeros((rows*cols,3), np.float32)
    objp[:,:2] = np.mgrid[0:rows,0:cols].T.reshape(-1,2)
    objp = objp * world_scaling
    width = image.shape[1]
    height = image.shape[0]   

    img_point = []
    points_3d = []
    gray = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
    criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 30, 0.001)
    ret, corners = cv.findChessboardCorners(gray, (rows,cols), None)
    
    if ret == True:
        corners2 = cv.cornerSubPix(gray, corners, (11,11), (-1,-1), criteria)
        img_point.append(corners2)
        points_3d.append(objp)
        cv.drawChessboardCorners(image, (rows,cols), corners2, ret)
        cv.imshow('image', image)
        cv.waitKey(0)
        cv.destroyAllWindows()
    else:
        print('corners not found')
        
    ret, mtx, dist, rvecs, tvecs = cv.calibrateCamera(points_3d,img_point, (width, height), None, None)
    print(ret)
    return mtx, dist, img_point, points_3d



In [7]:
mtx_R, dist_R, img_point_R, points_3d_R = find_carmera_matrix(im_right)
mtx_L, dist_L, img_point_L, points_3d_L = find_carmera_matrix(im_left)



# print(points_3d_R)
# print(points_3d_L)

2024-12-16 14:37:27.658 python[84483:6434995] +[IMKClient subclass]: chose IMKClient_Modern
2024-12-16 14:37:27.658 python[84483:6434995] +[IMKInputSession subclass]: chose IMKInputSession_Modern


0.12608556156016867
0.15063948980967823


In [8]:
print(dist_L)
print(dist_R)

width = im_left.shape[1]
height = im_left.shape[0]

[[-8.09780120e-03  8.02099058e-01 -1.20396192e-03 -5.19726747e-04
  -2.37977391e+00]]
[[-1.60761480e-01  4.87978787e+00 -3.53754258e-02 -1.01809029e-02
  -4.24485185e+01]]


In [9]:
def find_stereo_camera_matrix(mtx_L, dist_L, mtx_R, dist_R, img_point_L, img_point_R, points_3d):
    stereocalibration_flags = cv.CALIB_FIX_INTRINSIC
    criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 100, 0.0001)
    ret, CM1, dist1, CM2, dist2, R, T, E, F = cv.stereoCalibrate(points_3d, img_point_L, img_point_R, mtx_L, dist_L,
                                mtx_R, dist_R, (width, height), criteria = criteria, flags = stereocalibration_flags)
    return R, T


In [10]:
R,T = find_stereo_camera_matrix(mtx_L, dist_L, mtx_R, dist_R, img_point_L, img_point_R, points_3d_L)

In [11]:
RT1 = np.concatenate([np.eye(3), [[0],[0],[0]]], axis = -1)
P1 = mtx_L @ RT1

RT2 = np.concatenate([R, T], axis = -1)
P2 = mtx_R @ RT2


In [12]:
def find_center(M, shown = False):
    U, S, V = np.linalg.svd(M)
    if shown:
        print(S)
        print("Camera Center:", V[-1, :3] / V[-1, -1])
    return V[-1, :3] / V[-1, -1]

In [13]:
def DLT(P1, P2, point1, point2):
 
    A = [point1[1]*P1[2,:] - P1[1,:],
         P1[0,:] - point1[0]*P1[2,:],
         point2[1]*P2[2,:] - P2[1,:],
         P2[0,:] - point2[0]*P2[2,:]
        ]
    A = np.array(A).reshape((4,4))
    #print('A: ')
    #print(A)
 
    B = A.transpose() @ A
    from scipy import linalg
    U, s, Vh = linalg.svd(B, full_matrices = False)
 
    # print('Triangulated point: ')
    # print(Vh[3,0:3]/Vh[3,3])
    return Vh[3,0:3]/Vh[3,3]

In [14]:
pt = DLT(P1, P2, img_point_L[0][0][0], img_point_R[0][0][0])
pt = pt / pt[2]
print(pt)

[ 0.13019869 -0.15256168  1.        ]


In [15]:
pt = DLT(P1, P2, img_point_L[0][1][0], img_point_R[0][1][0])

In [16]:
c1 = find_center(P1)
c2 = find_center(P2)



In [17]:
def get_input_lines(im, min_lines=3):
    """
    Allows user to input line segments; computes centers and directions.
    Inputs:
        im: np.ndarray of shape (height, width, 3)
        min_lines: minimum number of lines required
    Returns:
        n: number of lines from input
        lines: np.ndarray of shape (3, n)
            where each column denotes the parameters of the line equation
        centers: np.ndarray of shape (3, n)
            where each column denotes the homogeneous coordinates of the centers
    """
    n = 0

    plt.figure()
    plt.imshow(im)
    plt.show()
    print('Set at least %d lines to compute vanishing point' % min_lines)

    clicked = plt.ginput(3, timeout=0, show_clicks=True)
    if not clicked or len(clicked) < 3:
        print('Need at least %d lines, you have %d now' % (min_lines, n))


        # Unpack user inputs and save as homogeneous coordinates
    pt1 = np.array([clicked[0][0], clicked[0][1], 1])
    pt2 = np.array([clicked[1][0], clicked[1][1], 1])
    pt3 = np.array([clicked[2][0], clicked[2][1], 1])

    return [pt1, pt2, pt3]

In [18]:
pts_left = (get_input_lines(im_left, 3))
pts_right = (get_input_lines(im_right, 3))

Set at least 3 lines to compute vanishing point
Set at least 3 lines to compute vanishing point


In [22]:
pts_x = []
pts_y = []
pts_z = []

for i in range(img_point_L[0].shape[0]):
    pt = DLT(P1, P2, img_point_L[0][i][0], img_point_R[0][i][0])
    # pt = pt / pt[2]
    pts_x.append(pt[0])
    pts_y.append(pt[1])
    pts_z.append(pt[2])
print(len(pts_x), len(pts_y), len(pts_z))


49 49 49


In [23]:
pt_table = np.array([DLT(P1, P2, pts_left[0], pts_right[0]), DLT(P1, P2, pts_left[1], pts_right[1]), DLT(P1, P2, pts_left[2], pts_right[2])])

v1 = pt_table[0] - pt_table[1]
v2 = pt_table[0] - pt_table[2]
normal = np.cross(v1, v2)

d = -np.dot(normal, pt_table[0])



In [24]:
fig = plt.figure()

ax = fig.add_subplot(111, projection='3d')

pts_x.append(c1[0]) 
pts_y.append(c1[1])
pts_z.append(c1[2])
pts_x.append(c2[0])
pts_y.append(c2[1])
pts_z.append(c2[2])

max_range = max([max(pts_x)-min(pts_x), max(pts_y)-min(pts_y), max(pts_z)-min(pts_z)])
mid_x = (max(pts_x) + min(pts_x)) * 0.5
mid_y = (max(pts_y) + min(pts_y)) * 0.5
mid_z = (max(pts_z) + min(pts_z)) * 0.5
ax.view_init(elev=135, azim=0)
ax.set_xlim(mid_x - max_range/2, mid_x + max_range/2)
ax.set_ylim(mid_y - max_range/2, mid_y + max_range/2)
ax.set_zlim(mid_z - max_range/2, mid_z + max_range/2)

ax.scatter(pts_x, pts_y, pts_z, c='b')  
ax.scatter(c1[0], c1[1], c1[2], c='r')
ax.scatter(c2[0], c2[1], c2[2], c='r')
ax.scatter(pt_table[0][0], pt_table[0][1], pt_table[0][2], c='g')
ax.scatter(pt_table[1][0], pt_table[1][1], pt_table[1][2], c='g')
ax.scatter(pt_table[2][0], pt_table[2][1], pt_table[2][2], c='g')
X_range = np.linspace(mid_x - max_range/2, mid_x + max_range/2, 10)
Y_range = np.linspace(mid_y - max_range/2, mid_y + max_range/2, 10)
n_x, n_y, n_z = normal
X, Y = np.meshgrid(X_range, Y_range)
Z = (-d - n_x * X - n_y * Y) / n_z
ax.plot_surface(X, Y, Z, alpha=0.5, color='cyan')
plt.show()

In [69]:
pts_y

[-0.15256167775411691,
 -0.1523152863002844,
 -0.1520907944933297,
 -0.15189291245246714,
 -0.15175363613667014,
 -0.15169712209893826,
 -0.15161653626529267,
 -0.12045195456685011,
 -0.12045617610507484,
 -0.12051100259472315,
 -0.12059046401640396,
 -0.12066724948050107,
 -0.12079559662974634,
 -0.12101742793655428,
 -0.0888667912988598,
 -0.08918434672859067,
 -0.0894036480674644,
 -0.08970452763351777,
 -0.09002628590314157,
 -0.09044428397856848,
 -0.0908036614925174,
 -0.057775951647410916,
 -0.058218464409112304,
 -0.0588248079953709,
 -0.05934851383854943,
 -0.05984772514458571,
 -0.06046673967312637,
 -0.061112976074812295,
 -0.02704901497205872,
 -0.027816482693198353,
 -0.02861002740520376,
 -0.02937525645908579,
 -0.03017395026006212,
 -0.03099983207696069,
 -0.03184904455259279,
 0.003115732586963575,
 0.002141953059729476,
 0.0011256619691906527,
 0.0001238554845392874,
 -0.0008889617369163392,
 -0.0019128181062623736,
 -0.0030085819517617312,
 0.032939383822440035,
 0.03